In [1]:
import pandas as pd
import geopandas as gd
import collections
import numpy as np

/var/folders/f_/xfklpqfj6kjg8yr6l1byph0h0000gn/T/ipykernel_5512/3043780728.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gd


In [2]:
# Importing dataset with house price and location
pd.set_option('display.max_columns', None)
df = pd.read_csv("data/properties-cdmx.csv")
df.head(5)

,datetime_created,datetime_modified,id,listing_id,title,bedrooms,bathrooms,parking_spaces,description,half_bathrooms,lot_size,construction_size,lot_length,lot_width,floors,floor,age,virtual_tour,public_url,property_type_id_id,source_id,_type,development_id,status,sub_type,agency_id,real_age,featured,datetime_created-2,datetime_modified-2,id-2,latitude,longitude,street,postal_code,settlement,town,state,show_exact_location,exterior_number,interior_number,properties_id,city,datetime_created-3,datetime_modified-3,id-3,type,amount,formated_amount,currency,unit,period,properties_id-2,monthly_payment
0,2022-06-01 23:14:25.490557+00,2022-07-13 08:01:49.897386+00,26054,NaN,casa en La Coruña Álamos,3.0,2.0,2.0,Vive en una de las colonias mejor ubicadas de ...,NaN,166.0,166.0,NaN,NaN,NaN,NaN,2.0,NaN,casa-en-la-coruna-alamos-icb-26054,3.0,NaN,NaN,NaN,I,NaN,11660,NaN,False,2022-06-01 23:14:25.50303+00,2022-06-01 23:14:25.503052+00,26054,19.401676,-99.141939,la coruña,03400,álamos,benito juárez,ciudad de méxico,NaN,125,NaN,26054,NaN,2022-06-01 23:14:25.604672+00,2022-07-13 08:01:49.892534+00,26053,NaN,5700000.0,NaN,MXN,NaN,NaN,26054,52191.0
1,2022-06-01 23:12:55.54002+00,2022-07-12 07:47:29.12493+00,25524,NaN,"departamento en Eje 5 Sur, San Antonio Ampliac...",2.0,2.0,1.0,"Sennse San Antonio, 95 es un proyecto de Senns...",NaN,84.0,84.0,NaN,NaN,NaN,NaN,1.0,NaN,departamento-en-eje-5-sur-san-antonio-ampliaci...,1.0,NaN,NaN,NaN,I,NaN,11654,NaN,False,2022-06-01 23:12:55.561255+00,2022-06-01 23:12:55.561281+00,25524,19.385737,-99.180502,"eje 5 sur, san antonio",03840,ampliación nápoles,benito juárez,ciudad de méxico,NaN,95,NaN,25524,NaN,2022-06-01 23:12:55.606874+00,2022-07-13 07:56:08.48646+00,25523,NaN,4704000.0,NaN,NaN,NaN,NaN,25524,43071.0
2,2022-05-04 09:52:20.872924+00,2022-07-12 07:47:29.376511+00,11762,EB-ES3852,oficina en insurgentes sur del valle norte ben...,0.0,0.0,0.0,Eficiente espacio acondicionadoMagnífica ubica...,0.0,0.0,140.0,0.0,0.0,0.0,8.0,0.0,NaN,oficina-en-insurgentes-sur-del-valle-norte-ben...,7.0,EB-ES3852,U,NaN,I,NaN,11628,0.0,False,2022-05-04 09:52:20.900822+00,2022-05-04 09:52:20.900848+00,11762,19.395444,-99.171549,Insurgentes sur,03100,del valle norte,benito juárez,ciudad de méxico,NaN,NaN,NaN,11762,ciudad de méxico,2022-05-04 09:52:21.006536+00,2022-07-13 07:56:08.777655+00,11762,rental,49000.0,"$49,000",MXN,total,NaN,11762,449.0
3,2022-06-01 23:13:37.986341+00,2022-07-13 07:58:07.489873+00,25778,NaN,departamento en Huatabampo Roma Sur,3.0,3.0,3.0,Excelente desarrollo en el corazón de la Roma ...,NaN,127.0,117.0,NaN,NaN,NaN,NaN,0.0,NaN,departamento-en-huatabampo-roma-sur-icb-25778,1.0,NaN,NaN,NaN,I,NaN,11666,NaN,False,2022-06-01 23:13:37.999065+00,2022-06-01 23:13:37.999084+00,25778,19.407117,-99.159436,huatabampo,06760,roma sur,cuauhtémoc,ciudad de méxico,NaN,30,NaN,25778,NaN,2022-06-01 23:13:38.092177+00,2022-07-13 07:58:07.484699+00,25777,NaN,7927000.0,NaN,MXN,NaN,NaN,25778,72582.0
4,2022-06-01 07:56:05.513708+00,2022-07-13 07:56:08.820352+00,21174,EB-KL8406,departamento en 16 de diciembre granjas navida...,2.0,2.0,1.0,10 departamentos de lujo ubicado a solo unos m...,0.0,0.0,131.0,0.0,0.0,0.0,5.0,2022.0,NaN,departamento-en-renta-con-opcion-a-compra-en-1...,1.0,EB-KL8406,D,NaN,I,NaN,11628,NaN,False,2022-06-01 07:56:05.538181+00,2022-06-01 07:56:05.538203+00,21174,19.372790,-99.282137,16 de Diciembre,05210,granjas navidad,cuajimalpa de morelos,ciudad de méxico,NaN,NaN,NaN,21174,ciudad de méxico,2022-06-01 07:56:05.742897+00,2022-07-13 07:56:08.813462+00,21174,sale,4270000.0,"$4,270,000",MXN,total,NaN,21174,39097.0


In [3]:
#Initial cleaning
dfx = df.copy()

# Drop properties which price is in USD or do not specify the currency
df = df[~(df["currency"].isnull() | df["currency"] == "USD")]

# Drop outliers
dfx = dfx[(dfx["amount"] > 500000) & (dfx["amount"] < 40000000)]
dfx["price"] = dfx["amount"]

#Dropping unwanted columns
drop_list = ["datetime_created", "datetime_modified", "listing_id", "title", "description" , "virtual_tour", "public_url", "source_id", "development_id", "agency_id", "datetime_created-2", "datetime_modified-2", "id-2", "street", "state", "show_exact_location", "exterior_number", "interior_number", "properties_id", "city", "_type", "sub_type", "formated_amount","type", "datetime_created-3", "datetime_modified-3", "id-3", "unit", "period", "properties_id-2","currency", "amount", "monthly_payment", "featured"]
dfx.drop(drop_list,axis=1,inplace = True)

#Drop duplicated ids
dfx.drop_duplicates(subset= ["id"], keep='first', inplace=True, ignore_index=False)

#Keeping just apartments and houses 1,3,4
types= [1,3,4]
dfx =dfx.loc[dfx["property_type_id_id"].isin(types)]

#Capping lot size and construction size
#dfx = dfx.loc[dfx["lot_size"] > 0]
dfx = dfx.loc[dfx["construction_size"] > 0]

#Dropping houses without coordinates
dfx = dfx[dfx['latitude'].notna()]
dfx = dfx[dfx['longitude'].notna()]

#Dropping houses without age
#dfx = dfx[dfx['real_age'].notna()]

#Fixing index
dfx.reset_index(inplace = True, drop = True)

In [4]:
#Importing houses features data
features = pd.read_csv("data/data-features.csv")
features.head()

,datetime_created,datetime_modified,id,listing_id,title,bedrooms,bathrooms,parking_spaces,description,half_bathrooms,lot_size,construction_size,lot_length,lot_width,floors,floor,age,virtual_tour,public_url,property_type_id_id,source_id,_type,development_id,status,sub_type,agency_id,real_age,featured,datetime_created-2,datetime_modified-2,id-2,name,category,properties_id
0,2022-03-31 06:12:02.468175+00,2022-07-13 08:00:22.908053+00,15,EB-HQ0775,departamento en colorines jesús del monte huix...,2.0,2.0,1.0,¿Te encantaría encontrar el departamento de tu...,0.0,0.0,100.0,NaN,NaN,0.0,0.0,1.0,NaN,departamento-en-renta-con-opcion-a-compra-en-c...,1.0,EB-HQ0775,D,NaN,I,NaN,11622.0,1.0,False,2022-03-31 06:12:02.580855+00,2022-03-31 06:12:02.580884+00,13,Balcón,NaN,15
1,2022-03-31 06:12:19.19036+00,2022-07-12 07:56:37.321847+00,20,EB-HQ3139,departamento en av. santo tomas santo tomas az...,2.0,2.0,1.0,¿Te encantaría encontrar el departamento de tu...,0.0,0.0,125.0,NaN,NaN,0.0,0.0,0.0,NaN,departamento-en-renta-con-opcion-a-compra-en-a...,1.0,EB-HQ3139,D,NaN,I,NaN,11622.0,0.0,False,2022-03-31 06:12:19.257323+00,2022-03-31 06:12:19.257343+00,18,Estacionamiento techado,NaN,20
2,2022-03-31 06:12:22.357097+00,2022-07-12 07:59:16.73783+00,21,EB-HJ6079,departamento en molino santa fe cuajimalpa cua...,2.0,2.0,2.0,¿Te encantaría encontrar el departamento de tu...,0.0,0.0,120.0,NaN,NaN,0.0,0.0,0.0,https://kuula.co/share/collection/7Y8HV?fs=1&v...,departamento-en-renta-con-opcion-a-compra-en-m...,1.0,EB-HJ6079,D,NaN,I,NaN,11622.0,0.0,False,2022-03-31 06:12:22.510543+00,2022-03-31 06:12:22.510564+00,19,Estacionamiento techado,NaN,21
3,2022-03-31 06:12:53.500822+00,2022-07-13 08:00:08.666968+00,28,EB-IP5870,departamento en lago alberto granada miguel hi...,2.0,2.0,0.0,¿Te encantaría encontrar el departamento de tu...,0.0,0.0,86.0,NaN,NaN,0.0,17.0,12.0,NaN,departamento-en-renta-con-opcion-a-compra-en-l...,1.0,EB-IP5870,D,NaN,I,NaN,11622.0,12.0,False,2022-03-31 06:12:53.58299+00,2022-03-31 06:12:53.583386+00,25,Estacionamiento techado,NaN,28
4,2022-03-31 06:13:13.464994+00,2022-07-13 07:59:23.585922+00,35,EB-IZ2166,departamento en alba insurgentes cuicuilco coy...,2.0,2.0,1.0,Clikalia te ofrece hermoso desarrollo en la co...,0.0,0.0,108.0,NaN,NaN,0.0,0.0,0.0,NaN,departamento-en-renta-con-opcion-a-compra-en-a...,1.0,EB-IZ2166,D,NaN,I,NaN,11622.0,0.0,False,2022-03-31 06:13:13.547228+00,2022-03-31 06:13:13.547248+00,31,Cisterna,NaN,35


In [5]:
names = features.name.unique()
names

array(['Balcón', 'Estacionamiento techado', 'Cisterna', 'Jardín',
       'Cuarto de servicio', 'Garaje', 'Aire acondicionado',
       'Roof garden', 'Seguridad 24 horas', 'Elevador',
       'Circuito cerrado', 'Acceso a la playa', 'Cocina integral',
       'Cocina equipada', 'Bodega', 'Alberca', 'Terraza',
       'Cancha de tenis', 'Gimnasio', 'Jacuzzi', 'Chimenea', 'Amueblado',
       'Área de parrilla', 'Calefacción', 'Cuarto de lavado', 'Cine',
       'Sala de fiestas', 'Seguridad', 'Almacén',
       'Estacionamiento para visitas', 'Patio', 'Zona de juegos',
       'Facilidad para estacionarse',
       'Accesibilidad para personas con discapacidad',
       'Fraccionamiento privado', 'Accesibilidad para adultos mayores',
       'Parrilla', 'Área de juegos infantiles', 'Cocina', 'Portero',
       'Vista al agua', 'Mascotas permitidas', 'Frente al agua',
       'Dos plantas', 'Estacionamiento', 'Acceso controlado',
       'Roof Garden', 'Áreas Verdes', 'Canchas', 'Ciclopista', 'Palapa'

In [6]:
from unidecode import unidecode

def clean_feature_name(text):
    text = text.lower()
    text = text.replace(' ', '_')
    return unidecode(text)

In [7]:
clean_names = [clean_feature_name(name) for name in names]
len(clean_names)

102

In [8]:
features_columns = pd.DataFrame(columns=clean_names)
features_columns

,balcon,estacionamiento_techado,cisterna,jardin,cuarto_de_servicio,garaje,aire_acondicionado,roof_garden,seguridad_24_horas,elevador,circuito_cerrado,acceso_a_la_playa,cocina_integral,cocina_equipada,bodega,alberca,terraza,cancha_de_tenis,gimnasio,jacuzzi,chimenea,amueblado,area_de_parrilla,calefaccion,cuarto_de_lavado,cine,sala_de_fiestas,seguridad,almacen,estacionamiento_para_visitas,patio,zona_de_juegos,facilidad_para_estacionarse,accesibilidad_para_personas_con_discapacidad,fraccionamiento_privado,accesibilidad_para_adultos_mayores,parrilla,area_de_juegos_infantiles,cocina,portero,vista_al_agua,mascotas_permitidas,frente_al_agua,dos_plantas,estacionamiento,acceso_controlado,roof_garden,areas_verdes,canchas,ciclopista,palapa,zona_comercial,casa_club,parque_acuatico,ubicacion,asador,area_infantil,pista_de_trote,gym_al_aire_libre,juegos_infantiles,transporte,parque,servicios_ocultos,escuelas,basketball,hospital,football,casa_de_cultura,conmutador,no_se_aceptan_mascotas,salon_de_usos_multiples,una_sola_planta,riego_por_aspersion,ecotecnologias,frente_a_la_playa,camaras_de_vigilancia,mini_super,salon_usos_multiples,spa,lavanderia,cancha_de_futbol,vigilancia,areas_verdes,circuito_cerrado_en_toda_la_calle,planta_electrica,bosque,seguridad_12_horas,parque_industrial,planta_baja,vista_al_mar,barda_perimetral,porton_electrico,area_de_gimnasio,parque_para_mascotas,permitido_fumar,pet_friendly,pista_de_jogging,alberca_techada_y_climatizada,privada,alarma,bar,canchas_de_tenis


In [9]:
# Match each feature with the property ID
for index, row in features.iterrows():
    if row.properties_id not in features_columns.index:
        new_row = [False for x in names]
        features_columns.loc[row.properties_id] = new_row
    
    feature_name = clean_feature_name(row['name'])
    
    features_columns.at[row.properties_id, feature_name] = True
    

In [10]:
df_features = features_columns.reset_index()
rename = {"index": "id"}
df_features.rename(columns=rename, inplace =True)
df_features

,id,balcon,estacionamiento_techado,cisterna,jardin,cuarto_de_servicio,garaje,aire_acondicionado,roof_garden,seguridad_24_horas,elevador,circuito_cerrado,acceso_a_la_playa,cocina_integral,cocina_equipada,bodega,alberca,terraza,cancha_de_tenis,gimnasio,jacuzzi,chimenea,amueblado,area_de_parrilla,calefaccion,cuarto_de_lavado,cine,sala_de_fiestas,seguridad,almacen,estacionamiento_para_visitas,patio,zona_de_juegos,facilidad_para_estacionarse,accesibilidad_para_personas_con_discapacidad,fraccionamiento_privado,accesibilidad_para_adultos_mayores,parrilla,area_de_juegos_infantiles,cocina,portero,vista_al_agua,mascotas_permitidas,frente_al_agua,dos_plantas,estacionamiento,acceso_controlado,roof_garden,areas_verdes,canchas,ciclopista,palapa,zona_comercial,casa_club,parque_acuatico,ubicacion,asador,area_infantil,pista_de_trote,gym_al_aire_libre,juegos_infantiles,transporte,parque,servicios_ocultos,escuelas,basketball,hospital,football,casa_de_cultura,conmutador,no_se_aceptan_mascotas,salon_de_usos_multiples,una_sola_planta,riego_por_aspersion,ecotecnologias,frente_a_la_playa,camaras_de_vigilancia,mini_super,salon_usos_multiples,spa,lavanderia,cancha_de_futbol,vigilancia,areas_verdes,circuito_cerrado_en_toda_la_calle,planta_electrica,bosque,seguridad_12_horas,parque_industrial,planta_baja,vista_al_mar,barda_perimetral,porton_electrico,area_de_gimnasio,parque_para_mascotas,permitido_fumar,pet_friendly,pista_de_jogging,alberca_techada_y_climatizada,privada,alarma,bar,canchas_de_tenis
0,15,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,20,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,21,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,28,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,35,False,False,True,False,False,False,F

In [11]:
# Merging the properties with its features
final_df = pd.merge(dfx,df_features)

#export to csv
final_df.to_csv("data/ES_mexico_houses_data.csv",  index= False)
final_df

,id,bedrooms,bathrooms,parking_spaces,half_bathrooms,lot_size,construction_size,lot_length,lot_width,floors,floor,age,property_type_id_id,status,real_age,latitude,longitude,postal_code,settlement,town,price,balcon,estacionamiento_techado,cisterna,jardin,cuarto_de_servicio,garaje,aire_acondicionado,roof_garden,seguridad_24_horas,elevador,circuito_cerrado,acceso_a_la_playa,cocina_integral,cocina_equipada,bodega,alberca,terraza,cancha_de_tenis,gimnasio,jacuzzi,chimenea,amueblado,area_de_parrilla,calefaccion,cuarto_de_lavado,cine,sala_de_fiestas,seguridad,almacen,estacionamiento_para_visitas,patio,zona_de_juegos,facilidad_para_estacionarse,accesibilidad_para_personas_con_discapacidad,fraccionamiento_privado,accesibilidad_para_adultos_mayores,parrilla,area_de_juegos_infantiles,cocina,portero,vista_al_agua,mascotas_permitidas,frente_al_agua,dos_plantas,estacionamiento,acceso_controlado,roof_garden,areas_verdes,canchas,ciclopista,palapa,zona_comercial,casa_club,parque_acuatico,ubicacion,asador,area_infantil,pista_de_trote,gym_al_aire_libre,juegos_infantiles,transporte,parque,servicios_ocultos,escuelas,basketball,hospital,football,casa_de_cultura,conmutador,no_se_aceptan_mascotas,salon_de_usos_multiples,una_sola_planta,riego_por_aspersion,ecotecnologias,frente_a_la_playa,camaras_de_vigilancia,mini_super,salon_usos_multiples,spa,lavanderia,cancha_de_futbol,vigilancia,areas_verdes,circuito_cerrado_en_toda_la_calle,planta_electrica,bosque,seguridad_12_horas,parque_industrial,planta_baja,vista_al_mar,barda_perimetral,porton_electrico,area_de_gimnasio,parque_para_mascotas,permitido_fumar,pet_friendly,pista_de_jogging,alberca_techada_y_climatizada,privada,alarma,bar,canchas_de_tenis
0,26054,3.0,2.0,2.0,NaN,166.0,166.0,NaN,NaN,NaN,NaN,2.0,3.0,I,NaN,19.401676,-99.141939,03400,álamos,benito juárez,5700000.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,25524,2.0,2.0,1.0,NaN,84.0,84.0,NaN,NaN,NaN,NaN,1.0,1.0,I,NaN,19.385737,-99.180502,03840,ampliación nápoles,benito juárez,4704000.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,25778,3.0,3.0,3.0,NaN,127.0,117.0,NaN,NaN,NaN,NaN,0.0,1.0,I,NaN,19.407117,-99.159436,06760,roma sur,cuauhtémoc,7927000.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,21174,2.0,2.0,1.0,0.0,0.0,131.0,0.0,0.0,0.0,5.0,2022.0,1.0,I,NaN,19.372790,-99.282137,05210,granja

In [12]:
english_df = final_df[["id", "bedrooms", "bathrooms","half_bathrooms","lot_size", "construction_size", "age", "property_type_id_id", "real_age", "latitude", "longitude", "postal_code", "settlement", "price", "balcon", "estacionamiento_techado", "cisterna", "jardin", "garaje","aire_acondicionado", "roof_garden", "seguridad_24_horas", "cocina_integral", "cocina_equipada","alberca","terraza", "amueblado", "cuarto_de_lavado", "cuarto_de_servicio", "seguridad", "patio", "facilidad_para_estacionarse","fraccionamiento_privado", "cocina","portero", "accesibilidad_para_personas_con_discapacidad", "estacionamiento", "acceso_controlado", "camaras_de_vigilancia", "seguridad_12_horas", "planta_baja"]].copy()

In [13]:
mapper = {
    "estacionamiento_techado":"parking_space_1",
    "balcon":"balcony",
    "cisterna":"water strorage",
    "jardin":"garden_1",
    "garaje":"parking_space_2",
    "aire_acondicionado":"air_conditioning",
    "seguridad_24_horas":"security_1",
    "cocina_integral":"kitchen_1",
    "cocina_equipada":"kitchen_2",
    "alberca":"pool",
    "terraza":"roof_garden_2",
    "amueblado":"furniture",
    "cuarto_de_lavado":"service_room_1",
    "cuarto_de_servicio":"service_room_2",
    "seguridad":"security_2",
    "patio":"garden_2",
    "facilidad_para_estacionarse":"parking_space_3",
    "fraccionamiento_privado":"gated_community_1",
    "cocina":"kitchen_3",
    "portero":"gated_community_2",
    "accesibilidad_para_personas_con_discapacidad":"accasible_for_handicap",
    "estacionamiento":"parking_space_4",
    "acceso_controlado":"gated_community_3",
    "camaras_de_vigilancia":"security_3",
    "seguridad_12_horas":"security_4",
    "planta_baja":"first_floor"   
}

In [14]:
english_df.rename(columns=mapper, inplace = True)

In [15]:
english_df["security"] = english_df["security_1"] | english_df["security_2"] | english_df["security_3"] | english_df["security_4"]
english_df["parking_space"] = english_df["parking_space_1"] | english_df["parking_space_2"] | english_df["parking_space_3"] | english_df["parking_space_4"]
english_df["kitchen"] = english_df["kitchen_1"] | english_df["kitchen_2"] | english_df["kitchen_3"]
english_df["garden"] = english_df["garden_1"] | english_df["garden_2"]
english_df["roof_Garden"] = english_df["roof_garden_2"]
english_df["service_room"] = english_df["service_room_1"] | english_df["service_room_2"]
english_df["gated_community"] = english_df["gated_community_1"] | english_df["gated_community_2"] | english_df["gated_community_3"]


In [16]:
english_df.drop(columns=["security_1", "security_2", "security_3", "security_4", "parking_space_1", "parking_space_2", "parking_space_3", "parking_space_4", "kitchen_1", "kitchen_2", "kitchen_3", "garden_1", "garden_2", "roof_garden_2", "roof_garden", "service_room_1", "service_room_2", "gated_community_1", "gated_community_2", "gated_community_3"], inplace = True)

In [17]:
# Set construction size to lot size to those properties that do not have this information
english_df["lot_size"] = english_df["lot_size"].fillna(english_df["construction_size"])
english_df["lot_size"] = np.where(english_df["lot_size"] == 0, english_df["construction_size"], english_df["lot_size"])

# Drop those houses that still do not have lot size
english_df = english_df[~english_df["lot_size"].isna()]

# Calculate the price per square meter for each property
english_df["price_per_area"] = english_df["price"] / english_df["lot_size"]

english_df["price_small"] = english_df["price"]/1000000
english_df["price_per_area_small"] = english_df["price_per_area"]/1000000

# Change NaN values in half_bathrooms to 0
english_df["half_bathrooms"] = english_df["half_bathrooms"].fillna(0)

display(english_df)

,id,bedrooms,bathrooms,half_bathrooms,lot_size,construction_size,age,property_type_id_id,real_age,latitude,longitude,postal_code,settlement,price,balcony,water strorage,air_conditioning,pool,furniture,accasible_for_handicap,first_floor,security,parking_space,kitchen,garden,roof_Garden,service_room,gated_community,price_per_area,price_small,price_per_area_small
0,26054,3.0,2.0,0.0,166.0,166.0,2.0,3.0,NaN,19.401676,-99.141939,03400,álamos,5700000.0,False,False,False,False,False,False,False,True,False,False,True,False,True,False,34337.349398,5.700000,0.034337
1,25524,2.0,2.0,0.0,84.0,84.0,1.0,1.0,NaN,19.385737,-99.180502,03840,ampliación nápoles,4704000.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,56000.000000,4.704000,0.056000
2,25778,3.0,3.0,0.0,127.0,117.0,0.0,1.0,NaN,19.407117,-99.159436,06760,roma sur,7927000.0,False,False,False,False,False,False,False,True,False,False,True,False,False,False,62417.322835,7.927000,0.062417
3,21174,2.0,2.0,0.0,131.0,131.0,2022.0,1.0,NaN,19.372790,-99.282137,05210,granjas navidad,4270000.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,32595.419847,4.270000,0.032595
4,12469,2.0,2.0,0.0,91.0,91.0,0.0,1.0,0.0,19.362124,-99.287678,05320,cuajimalpa,5155286.0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,56651.494505,5.155286,0.056651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11277,13420,6.0,7.0,3.0,117.0,411.0,1.0,3.0,1.0,19.394592,-99.218025,01110,belén,14000000.0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,119658.119658,14.000000,0.119658
11278,2097,3.0,3.0,0.0,255.0,255.0,2.0,1.0,2.0,19.341719,-99.229973,01700,tetelpan,10050000.0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,39411.764706,10.050000,0.039412
11279,7567,3.0,2.0,0.0,90.0,90.0,11.0,1.0,11.0,19.297200,-99.141280,14370,villa lázaro cárdenas,3500000.0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,38888.888889,3.500000,0.038889
11280,1895,3.0,2.0,1.0,167.0,252.0,18.0,3.0,18.0,19.311713,-99.244771,10610,el toro,6295000.0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,37694.610778,6.295000,0.037695


In [18]:
# converting 'age' from float to int
english_df["age"] = english_df["age"].astype(int)
english_df["real_age"] = english_df.apply(lambda x: 2022 - x.age if len(str(x.age)) == 4 else x.age, axis=1)

display(english_df)

,id,bedrooms,bathrooms,half_bathrooms,lot_size,construction_size,age,property_type_id_id,real_age,latitude,longitude,postal_code,settlement,price,balcony,water strorage,air_conditioning,pool,furniture,accasible_for_handicap,first_floor,security,parking_space,kitchen,garden,roof_Garden,service_room,gated_community,price_per_area,price_small,price_per_area_small
0,26054,3.0,2.0,0.0,166.0,166.0,2,3.0,2,19.401676,-99.141939,03400,álamos,5700000.0,False,False,False,False,False,False,False,True,False,False,True,False,True,False,34337.349398,5.700000,0.034337
1,25524,2.0,2.0,0.0,84.0,84.0,1,1.0,1,19.385737,-99.180502,03840,ampliación nápoles,4704000.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,56000.000000,4.704000,0.056000
2,25778,3.0,3.0,0.0,127.0,117.0,0,1.0,0,19.407117,-99.159436,06760,roma sur,7927000.0,False,False,False,False,False,False,False,True,False,False,True,False,False,False,62417.322835,7.927000,0.062417
3,21174,2.0,2.0,0.0,131.0,131.0,2022,1.0,0,19.372790,-99.282137,05210,granjas navidad,4270000.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,32595.419847,4.270000,0.032595
4,12469,2.0,2.0,0.0,91.0,91.0,0,1.0,0,19.362124,-99.287678,05320,cuajimalpa,5155286.0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,56651.494505,5.155286,0.056651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11277,13420,6.0,7.0,3.0,117.0,411.0,1,3.0,1,19.394592,-99.218025,01110,belén,14000000.0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,119658.119658,14.000000,0.119658
11278,2097,3.0,3.0,0.0,255.0,255.0,2,1.0,2,19.341719,-99.229973,01700,tetelpan,10050000.0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,39411.764706,10.050000,0.039412
11279,7567,3.0,2.0,0.0,90.0,90.0,11,1.0,11,19.297200,-99.141280,14370,villa lázaro cárdenas,3500000.0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,38888.888889,3.500000,0.038889
11280,1895,3.0,2.0,1.0,167.0,252.0,18,3.0,18,19.311713,-99.244771,10610,el toro,6295000.0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,37694.610778,6.295000,0.037695


In [19]:
# Drop properties that do not have information about bedrooms and bathrooms
english_df = english_df[~english_df["bedrooms"].isna()]
english_df = english_df[~english_df["bathrooms"].isna()]
english_df = english_df[english_df["bathrooms"] != 0]

In [20]:
a = english_df.columns.to_list()
print([item for item, count in collections.Counter(a).items() if count > 1])

[]


In [21]:
# Drop the duplicate columns
english_df = english_df.loc[:, ~english_df.columns.duplicated()]

In [22]:
a = english_df.columns.to_list()
print([item for item, count in collections.Counter(a).items() if count > 1])

[]


In [23]:
english_df.to_csv("data/EN_merged_mexico_houses_data.csv",  index= False)

In [24]:
# Convert to geodataframe
gdf = gd.GeoDataFrame(english_df, geometry=gd.points_from_xy(english_df.longitude, english_df.latitude))
gdf.head()

,id,bedrooms,bathrooms,half_bathrooms,lot_size,construction_size,age,property_type_id_id,real_age,latitude,longitude,postal_code,settlement,price,balcony,water strorage,air_conditioning,pool,furniture,accasible_for_handicap,first_floor,security,parking_space,kitchen,garden,roof_Garden,service_room,gated_community,price_per_area,price_small,price_per_area_small,geometry
0,26054,3.0,2.0,0.0,166.0,166.0,2,3.0,2,19.401676,-99.141939,03400,álamos,5700000.0,False,False,False,False,False,False,False,True,False,False,True,False,True,False,34337.349398,5.700000,0.034337,POINT (-99.14194 19.40168)
1,25524,2.0,2.0,0.0,84.0,84.0,1,1.0,1,19.385737,-99.180502,03840,ampliación nápoles,4704000.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,56000.000000,4.704000,0.056000,POINT (-99.18050 19.38574)
2,25778,3.0,3.0,0.0,127.0,117.0,0,1.0,0,19.407117,-99.159436,06760,roma sur,7927000.0,False,False,False,False,False,False,False,True,False,False,True,False,False,False,62417.322835,7.927000,0.062417,POINT (-99.15944 19.40712)
3,21174,2.0,2.0,0.0,131.0,131.0,2022,1.0,0,19.372790,-99.282137,05210,granjas navidad,4270000.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,32595.419847,4.270000,0.032595,POINT (-99.28214 19.37279)
4,12469,2.0,2.0,0.0,91.0,91.0,0,1.0,0,19.362124,-99.287678,05320,cuajimalpa,5155286.0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,56651.494505,5.155286,0.056651,POINT (-99.28768 19.36212)


In [25]:
# Set the CRS of the GeoDataFrame to EPSG code 3857 (Web Mercator or Google Mercator) because the dataset uses the coordinates from that CRS
gdf.crs = {'init': 'epsg:4326'}
gdf.crs

/Users/abril/opt/anaconda3/envs/ssml/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [26]:
gdf.to_file("data/EN-Merged-properties-cdmx.gpkg", driver = "GPKG")